In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import notebook
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from tqdm import tqdm
from hloc.utils.read_write_model import read_images_binary,read_points3D_binary
from PIL import Image
from scipy.interpolate import griddata
import pickle
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import notebook
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from tqdm import tqdm
from hloc.utils.read_write_model import read_images_binary,read_points3D_binary
from PIL import Image
from scipy.interpolate import griddata
from tqdm import tqdm
from hloc.utils.read_write_model import read_images_binary,read_points3D_binary,read_cameras_binary,qvec2rotmat
from PIL import Image
from scipy.interpolate import griddata
import cv2
from pathlib import Path
import torch
import pickle


In [3]:
images_path='outputs/aachen/sfm_superpoint+superglue/images.bin'
points3Ds_path='outputs/aachen/sfm_superpoint+superglue/points3D.bin'
caperas_path='outputs/aachen/sfm_superpoint+superglue/cameras.bin'
images = read_images_binary(images_path)
points3Ds = read_points3D_binary(points3Ds_path)
cameras = read_cameras_binary(caperas_path)

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "segment-anything-main/sam_vit_h_4b8939.pth"
model_type = "vit_h"

In [4]:
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [5]:
def extract_patch_2D_points(xys, img_width, img_height, size):
    patch_width = img_width // size  # 16x16 patches to get 256 patches
    patch_height = img_height // size

    patches_2D_points = {}
    for i in range(size):
        for j in range(size):
            # Define patch boundaries
            x_start, x_end = i * patch_width, (i + 1) * patch_width
            y_start, y_end = j * patch_height, (j + 1) * patch_height
            
            # Find 2D points inside the patch
            mask = (xys[:, 0] >= x_start) & (xys[:, 0] < x_end) & (xys[:, 1] >= y_start) & (xys[:, 1] < y_end)
            
            # Get the 2D points for the points inside the patch
            patch_2D_points = xys[mask]
            
            # Store in the dictionary
            patches_2D_points[(j, i)] = patch_2D_points.tolist()

    return patches_2D_points


In [6]:
continue_img_id=[1167,3266,3302,3297]
# For each image in the modified images dictionary
iter=0
small=7
large =4
Pre_Small= 20
Pre_Large = 14
size=Pre_Large
TotalImageKey = {}
TotalImageKey3D = {}
iter=0

In [7]:
for image_id, image in tqdm(images.items()):
    xys = image.xys
    point3D_ids = image.point3D_ids

    # Get the image dimensions
    img = Image.open('datasets/aachen/images/images_upright/'+image.name)
    img_width, img_height = img.size

    # Extract 3D ids for each patch in the image
    patches_2D_ids = extract_patch_2D_points(xys, img_width, img_height, size)
    patches_2D_ids=dict(list(patches_2D_ids.items())) 
    ########3
    key = image_id
    tar_img = images[key]
    tar_cameras = cameras[key]
    tar_camera_header = [tar_cameras.id, tar_cameras.width, tar_cameras.height, *tar_cameras.params]
    tar_image_header = [tar_img.id, *tar_img.qvec, *tar_img.tvec, tar_img.camera_id, tar_img.name]
    tar_points_strings = []

    # for xy, point3D_id in zip(tar_img.xys, tar_img.point3D_ids):
    #     if(point3D_id == -1):
    #         continue
    #     tar_points_strings.append(" ".join(map(str, [*xy, point3D_id])))



    image_dir =Path('datasets/aachen/images/images_upright/')
    path = image_dir / tar_image_header[9]
    image = cv2.imread(str(path))


    # #for str_points in tar_points_strings:
    TopSky=[]
    ElseKey=[]
    for key, patches in patches_2D_ids.items():
        if not patches:            
            # Compute the center of the patch
            #print(key)
            patch_center_x = key[1] * (img_width // size) + (img_width // (2*size))
            patch_center_y = key[0] * (img_height // size) + (img_height // (2*size))
            if key[0]==0:
                TopSky.append([patch_center_x,patch_center_y])
        ##여기에 추가적으로 아무 값이 없는것은 lable을 추가해 주어야 한다. 
    ImageKey = {}
    if len(TopSky)==0:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        predictor.set_image(image)
        input_points_AllBuilding=[]
        patch_info={}
        for key, patches in patches_2D_ids.items():
            x_start, x_end = key[1] * (img_width // size), (key[1] + 1) * (img_width // size)
            y_start, y_end = key[0] * (img_height // size), (key[0] + 1) * (img_height // size)

            patch_center_x = key[1] * (img_width // size) + (img_width // (2*size))
            patch_center_y = key[0] * (img_height // size) + (img_height // (2*size))
            
            patch_info[key] = {
                "x_start": x_start, "x_end": x_end,
                "y_start": y_start, "y_end": y_end,
                "center": [patch_center_x, patch_center_y]
            }
            patch_data = patch_info[key]
            input_points_AllBuilding.append(patch_data["center"])
            ImageKey[key]=patch_data["center"]
        
        input_points_AllBuilding = np.array(input_points_AllBuilding)
        input_labels_AllBuilding = np.zeros(len(input_points_AllBuilding),dtype=int)

        #plt.figure(figsize=(5,5))
        #plt.imshow(image)
        #show_points(input_points_AllBuilding, input_labels_AllBuilding, plt.gca())
        #plt.axis('off')
        #filename = "NoMask.png"
        #imgname=str(tar_image_header[9]).split('.')[0].split('/')[-1]
        #filename = "Segmentation/MultiMaskAll4x4_09292035/"+imgname+"_"+filename
        #plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    else:
        #하늘이 없는 이미지에도 패치를 그려야 한다. 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        predictor.set_image(image)

        input_points_TopSky = np.array(TopSky)
        input_labels_TopSky = np.ones(len(TopSky),dtype=int)

        masks, scores, logits = predictor.predict(
        point_coords=input_points_TopSky,
        point_labels=input_labels_TopSky,
        multimask_output=True,
        )
        imgname=str(tar_image_header[9]).split('.')[0].split('/')[-1]
        for i, (mask, score) in enumerate(zip(masks, scores)):
            if i == 1:
                patch_info={}
                input_points_YesSky=[]
                input_points_NoSky=[]
                for key, patches in patches_2D_ids.items():
                    x_start, x_end = key[1] * (img_width // size), (key[1] + 1) * (img_width // size)
                    y_start, y_end = key[0] * (img_height // size), (key[0] + 1) * (img_height // size)

                    patch_center_x = key[1] * (img_width // size) + (img_width // (2*size))
                    patch_center_y = key[0] * (img_height // size) + (img_height // (2*size))
                    
                    patch_info[key] = {
                        "x_start": x_start, "x_end": x_end,
                        "y_start": y_start, "y_end": y_end,
                        "center": [patch_center_x, patch_center_y]
                    }
                    patch_data = patch_info[key]
                    patch_mask = masks[0, patch_data["y_start"]:patch_data["y_end"], patch_data["x_start"]:patch_data["x_end"]]
                    
            # 패치 내의 픽셀 중 90% 이상이 True인지 검사
                    true_percentage = np.sum(patch_mask) / (patch_mask.shape[0] * patch_mask.shape[1])
                    
                    if true_percentage >= 0.9:
                        input_points_YesSky.append(patch_data["center"])
                        ImageKey[key]=[0,0]
                    else:
                        input_points_NoSky.append(patch_data["center"])
                        ImageKey[key]=patch_data["center"]

                input_points_YesSky = np.array(input_points_YesSky)
                input_labels_YesSky = np.ones(len(input_points_YesSky),dtype=int)

                input_points_NoSky = np.array(input_points_NoSky)
                input_labels_NoSky = np.zeros(len(input_points_NoSky),dtype=int)
                if len(input_labels_YesSky) > 0:
                    input_points = np.vstack((input_points_YesSky, input_points_NoSky))
                    input_labels = np.concatenate((input_labels_YesSky, input_labels_NoSky))



                #plt.figure(figsize=(5,5))
                #plt.imshow(image)
                #show_mask(mask, plt.gca())
                #show_points(input_points, input_labels, plt.gca())
                #plt.axis('off')
                #filename = f"mask_{i+1}_score_{score:.3f}.png"
                #filename = "Segmentation/MultiMaskAll4x4_09292035/"+imgname+"_"+filename
                #plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    ImageKey = {key: ImageKey[key] for key in sorted(ImageKey.keys(), key=lambda x: (x[0], x[1]))}
    TotalImageKey[image_id]=ImageKey
    #3D Point 변환
    tar_quat = np.array(tar_image_header[1:5])
    rvec=qvec2rotmat(tar_quat)
    tar_trans = np.array(tar_image_header[5:8])
    camera_metrix = np.zeros((3, 3), dtype='float32')
    camera_metrix[0, 0] = tar_camera_header[3]
    camera_metrix[1, 1] = tar_camera_header[3]
    camera_metrix[0, 2] = tar_camera_header[4]
    camera_metrix[1, 2] = tar_camera_header[5]
    camera_metrix[2, 2] = 1
    distort=tar_camera_header[6]
    ImageKey3D={}
    ImageKey3D['Token'] = images[image_id].tvec
    for key, patches in TotalImageKey[image_id].items():
        if patches ==[0,0]:
            #ImageKey3D[key]=[0,0,0]
            #ImageKey3D[key]=np.zeros(3)
            ImageKey3D[key]=np.array([100000,100000,100000])
        else:
            target_img = images[image_id]
            target_cam = cameras[target_img.camera_id]
            K = np.array([
                [target_cam.params[0], 0, target_cam.params[1]],
                [0, target_cam.params[0], target_cam.params[2]],
                [0, 0, 1]
            ])
            R = qvec2rotmat(target_img.qvec)
            t = target_img.tvec.reshape(3, 1)
            # RT = np.hstack([R, t])
            RT_4x4 = np.eye(4)
            RT_4x4[:3, :3] = R
            RT_4x4[:3, 3] = t.reshape(-1)
            RT=RT_4x4
            point_2d = np.array([int(patches[0]), int(patches[1]), 1]).reshape(3, 1)
            point_3d_cam = np.linalg.inv(K).dot(point_2d)
            point_3d_cam = point_3d_cam / np.linalg.norm(point_3d_cam)  # Normalize to keep it at unit distance

            # Transforming the 3D point in camera coordinate system to world coordinate system
            point_3d_world = np.linalg.inv(RT).dot(np.vstack([point_3d_cam * 10, 1]))  # Multiplying by 10 to keep the depth arbitrary
            point_3d_world = point_3d_world[:3].reshape(-1)
            ImageKey3D[key]=point_3d_world
    TotalImageKey3D[image_id]=ImageKey3D
    iter+=1

  0%|          | 0/4328 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
with open("DataBase/TotalImageKey3D_14x14.pickle", "wb") as file:
    pickle.dump(TotalImageKey3D, file)

In [8]:
# to load pickle file
with open('DataBase/TotalImageKey3D_14x14.pickle', 'rb') as f:
    all_representative_points = pickle.load(f)


In [9]:

for key in tqdm(images.keys()):
    # 각 키에 대응하는 3D 점을 가져옴
    points = all_representative_points[key]

    # images[key].qvec 값을 가져옴
    qvec = torch.tensor(images[key].qvec)

    # 사전의 각 점에 대해 qvec 추가
    for point_key in points:
        # 현재 점을 PyTorch 텐서로 변환
        current_point = torch.tensor(points[point_key])

        # 현재 점과 qvec 연결
        concatenated_point = torch.cat((current_point, qvec), dim=0)

        # 업데이트된 값을 사전에 저장
        points[point_key] = concatenated_point

    # 업데이트된 사전을 원본 사전에 저장
    all_representative_points[key] = points


  0%|          | 0/4328 [00:00<?, ?it/s]

In [10]:
with open('DataBase/TotalImageKeyRT_14x14.pickle', 'wb') as f:
    pickle.dump(all_representative_points, f, pickle.HIGHEST_PROTOCOL)

In [11]:
all_representative_points[630]['Token']

tensor([ 2.5375e+02, -1.2629e+02, -5.7944e+02,  5.0160e-02,  5.5191e-01,
         9.5616e-02,  8.2689e-01], dtype=torch.float64)

In [12]:
import torch

def rotary_position_embeddings_7d(coords, dim):
    # Normalize coordinates to [-1, 1]
    # Normalize coordinates to [-1, 1]
    coords = 2.0 * (coords - coords.min()) / (coords.max() - coords.min()) - 1.0

    # Convert normalized coordinates to angles in range [-pi, pi]
    angles = coords * torch.pi

    # Compute sin and cos for each dimension
    sin = torch.sin(angles)
    cos = torch.cos(angles)

    # Create embeddings by interleaving sin and cos values
    embeddings = torch.zeros(*coords.shape[:-1], dim)

    # Calculate the number of complete sin-cos pairs that can fit in the embeddings
    full_pairs = dim // 14

    for i in range(full_pairs):
        embeddings[..., 14*i:14*i+7] = sin
        embeddings[..., 14*i+7:14*i+14] = cos

    # Handle the case where dim is not a multiple of 14
    remaining_dims = dim % 14
    if remaining_dims > 0:
        # Calculate the repeat count for each dimension
        sin_repeat_count = [1] * (sin.dim() - 1) + [(remaining_dims + 6) // 7]
        cos_repeat_count = [1] * (cos.dim() - 1) + [(remaining_dims + 6) // 7]

        # Repeat sin and cos to fill the remaining dimensions
        extended_sin = sin.repeat(*sin_repeat_count)
        extended_cos = cos.repeat(*cos_repeat_count)

        if remaining_dims <= 7:
            embeddings[..., -remaining_dims:] = extended_sin[..., :remaining_dims]
        else:
            embeddings[..., -remaining_dims:-7] = extended_sin[..., :remaining_dims-7]
            embeddings[..., -7:] = extended_cos[..., :(remaining_dims-7)]

    return embeddings


In [13]:
import torch

def rotary_position_embeddings_7d_to_192d(coords):
    """
    Compute Rotary Position Embeddings for 7D coordinates, extending them to 192 dimensions.

    Args:
    coords (Tensor): A tensor of shape (..., 7) representing 7D coordinates.

    Returns:
    Tensor: A tensor of rotary position embeddings with 192 dimensions.
    """
    dim = 192
    # Normalize coordinates to [-1, 1]
    coords = 2.0 * (coords - coords.min()) / (coords.max() - coords.min()) - 1.0

    # Convert normalized coordinates to angles in range [-pi, pi]
    angles = coords * torch.pi

    # Compute sin and cos for each dimension
    sin = torch.sin(angles)
    cos = torch.cos(angles)

    # Initialize the embeddings tensor
    embeddings = torch.zeros(*coords.shape[:-1], dim)

    # Determine the number of complete 14-dimensional (7 sin + 7 cos) blocks
    full_blocks = dim // 14

    # Fill the embeddings tensor with repeated sin and cos blocks
    for i in range(full_blocks):
        embeddings[..., 14*i:14*i+7] = sin
        embeddings[..., 14*i+7:14*i+14] = cos

    # Handle remaining dimensions if dim is not a multiple of 14
    remaining_dims = dim % 14
    if remaining_dims > 0:
        extended_sin_cos = torch.cat((sin, cos), dim=-1).flatten()
        embeddings[..., -remaining_dims:] = extended_sin_cos[:remaining_dims]

    return embeddings


In [14]:
import torch
dim = 192
include_input=True
encoded_points = {}
include_input = True
for id, reps in tqdm(all_representative_points.items()):
    encoded_points[id] = {}
    for key, value in reps.items():
        if value is not None:
            input = torch.tensor(value)
            input = input.unsqueeze(0)
            output = rotary_position_embeddings_7d_to_192d(input)
            encoded_points[id][key] = torch.tensor(output.squeeze(0).tolist())

  0%|          | 0/4328 [00:00<?, ?it/s]

/tmp/ipykernel_38912/3292865358.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(value)


In [15]:
tensor_dict = {}
for key in tqdm(encoded_points):
    token_value = encoded_points[key]['Token']
    patch_values = [encoded_points[key][patch_key] for patch_key in encoded_points[key] if patch_key != 'Token']

    token_tensor = torch.tensor(token_value)
    patch_tensors = [torch.tensor(patch_value) for patch_value in patch_values]

    # torch.stack을 사용하여 텐서들을 쌓습니다.
    tensor_array = torch.stack([token_tensor] + patch_tensors)

    tensor_dict[key] = tensor_array


  0%|          | 0/4328 [00:00<?, ?it/s]

/tmp/ipykernel_38912/1490317307.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_tensor = torch.tensor(token_value)
/tmp/ipykernel_38912/1490317307.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_tensors = [torch.tensor(patch_value) for patch_value in patch_values]


In [16]:
with open('DataBase/Large_Patch_14x14_RT_RoPE_Tensor.pickle', 'wb') as f:
    pickle.dump(tensor_dict, f, pickle.HIGHEST_PROTOCOL)

# 여기까지